In [ ]:
#!pip install dash
#!pip install dash_bootstrap_components

In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

In [ ]:
#Loading the dataset
df = pd.read_csv("/content/Marketing_Campaign_After_Clean.csv")
df.head()

,Education,Income,Recency,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,Loyalty,Age,Children,Spent,TotalAcceptedCmp,MaritalStatus,Family_Size
0,Graduate,58138.0,58,3,8,10,4,7,0,1,605,57,0,1617,0,Single,1
1,Graduate,46344.0,38,2,1,1,2,5,0,0,75,60,2,27,0,Single,3
2,Graduate,71613.0,26,1,8,2,10,4,0,0,286,49,0,776,0,Married,2
3,Graduate,26646.0,26,2,2,0,4,6,0,0,113,30,1,53,0,Married,3
4,Postgraduate,58293.0,94,5,5,3,6,5,0,0,67,33,1,422,0,Married,3


In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr(numeric_only=True)

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])

# Layout of the Dashboard
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col( html.H1(
        "Customer Segmentation Dashboard",
        className="text-center mb-4",
        style={'color': '#7FCDBB'}
    ),  width=12)
    ]),
    dbc.Row([
        dbc.Col([
            html.Label('Select Education Level'),
            dcc.Dropdown(
                id='education-filter',
                options=[{'label': i, 'value': i} for i in df['Education'].unique()],
                value=None,
                placeholder='Filter by Education',
            ),
            html.Br(),
            html.Label('Select Marital Status'),
            dcc.Dropdown(
                id='marital-status-filter',
                options=[{'label': i, 'value': i} for i in df['MaritalStatus'].unique()],
                value=None,
                placeholder='Filter by Marital Status',
            ),
        ], width=4),

        dbc.Col([
            html.Label('Income Range'),
            dcc.RangeSlider(
                id='income-slider',
                min=df['Income'].min(),
                max=df['Income'].max(),
                step=1000,
                value=[df['Income'].min(), df['Income'].max()],
                marks={i: f"${i}" for i in range(int(df['Income'].min()), int(df['Income'].max()) + 1, 10000)},
                tooltip={"placement": "bottom", "always_visible": True},
                updatemode='drag',
            ),
            html.Br(),
            html.Label('Age Range'),
            dcc.RangeSlider(
                id='age-slider',
                min=df['Age'].min(),
                max=df['Age'].max(),
                step=1,
                value=[df['Age'].min(), df['Age'].max()],
                marks={i: str(i) for i in range(int(df['Age'].min()), int(df['Age'].max()) + 1, 5)},
                tooltip={"placement": "bottom", "always_visible": True},
                updatemode='drag'
            ),
        ], width=8),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([dcc.Graph(id='scatter-plot')], width=12)
    ]),
    html.Br(),
    html.Br(),
    dbc.Row([
    dbc.Col([
            html.Label('Select Column for Bar and Pie Charts'),
            dcc.Dropdown(
                id='chart-column-selector',
                options=[{'label': 'Response', 'value': 'Response'},
                         {'label': 'Children', 'value': 'Children'},
                         {'label': 'Family_Size', 'value': 'Family_Size'},
                         {'label': 'Complain', 'value': 'Complain'},
                         ],
                value='Children',  # Default value
                placeholder='Select a column'
            ),
        ], width=4),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([dcc.Graph(id='bar-chart')], width=6),
        dbc.Col([dcc.Graph(id='pie-chart')], width=6)
    ]),
    html.Br(),
    html.Br(),
    dbc.Row([
      dbc.Col([
              html.Label('Select Column for Histogram and BoxPlot'),
              dcc.Dropdown(
                  id='hist-column-selector',
                  options=[{'label': 'Recency', 'value': 'Recency'},
                          {'label': 'NumWebVisitsMonth', 'value': 'NumWebVisitsMonth'},
                          {'label': 'NumDealsPurchases', 'value': 'NumDealsPurchases'},
                          ],
                  value='Recency',  # Default value
                  placeholder='Select a column'
              ),
          ], width=4),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([dcc.Graph(id='histogram')], width=6),
        dbc.Col([dcc.Graph(id='boxplot')], width=6)
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id='heatmap',
                figure=px.imshow(
                    correlation_matrix,
                    text_auto=True,
                    color_continuous_scale='Aggrnyl',
                    title='Correlation Matrix',
                ).update_layout(
                    title_x=0.5,
                    title_font=dict(size=20, family='Arial', color='black'),  # Title font customization
                    width=1400,  # Width of the heatmap
                    height=1400,  # Height of the heatmap
                    xaxis=dict(tickangle=45),  # Rotate x-axis labels
                    yaxis=dict(tickangle=0),  # Keep y-axis labels horizontal
                )
            )
        ], width=12),
    ]),
        dbc.Row([
        dbc.Col([
            dcc.Graph(
                id='pie-chart-total-purchases',
                figure={
                    'data': [{
                        'labels': ['Web Purchases', 'Catalog Purchases', 'Store Purchases'],
                        'values': [df['NumWebPurchases'].sum(), df['NumCatalogPurchases'].sum(), df['NumStorePurchases'].sum()],
                        'type': 'pie',
                        'marker': {'colors': px.colors.sequential.Aggrnyl_r },
                    }],
                    'layout': {
                        'title': 'Percentage of Total Purchases by Channel',
                        'showlegend': True,
                        'template': 'plotly_dark',
                        'title_font': dict(size=20, family='Arial', color='black'),
                    }
                }
            )
        ], width=12),
    ]),

])

# Callback to update the scatter plot, bar chart, and pie chart based on filters
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-chart', 'figure'),
     Output('pie-chart', 'figure'),
     Output('histogram', 'figure'),
     Output('boxplot', 'figure')],
    [
     Input('education-filter', 'value'),
     Input('marital-status-filter', 'value'),
     Input('income-slider', 'value'),
     Input('age-slider', 'value'),
     Input('chart-column-selector', 'value'),
     Input('hist-column-selector', 'value')
    ]
)
def update_graph(education, marital_status, income_range, age_range, selected_column,hist_column):
    # Filter the dataframe based on the inputs
    filtered_df = df.copy()

    if education:
        filtered_df = filtered_df[filtered_df['Education'] == education]

    if marital_status:
        filtered_df = filtered_df[filtered_df['MaritalStatus'] == marital_status]

    filtered_df = filtered_df[
        (filtered_df['Income'] >= income_range[0]) & (filtered_df['Income'] <= income_range[1]) &
        (filtered_df['Age'] >= age_range[0]) & (filtered_df['Age'] <= age_range[1])
    ]

    # Scatter plot for Income vs. Spent, with Age as the color
    scatter_fig = px.scatter(
        filtered_df, x='Income', y='Spent', color='Age',
        title="Income vs. Spent by Age",
        labels={'Income': 'Income ($)', 'Spent': 'Spent ($)', 'Age': 'Age'},
        template='plotly_dark',
        color_continuous_scale='Aggrnyl',  # Use a continuous color scale
    )

    # Count occurrences of each category
    counts = filtered_df[selected_column].value_counts()
    # Bar Chart
    bar_fig = px.bar(
        x=counts.index,
        y=counts.values,
        labels={'x': selected_column, 'y': 'Count'},
        title=f'Bar Chart for {selected_column}',
        template='plotly_dark',
        color=counts.index,  # Set color based on categories
        #color_discrete_sequence=px.colors.sequential.Aggrnyl_r # Use a qualitative color sequence
        color_continuous_scale='Aggrnyl'
    )

    # Pie Chart
    pie_fig = px.pie(
        values=counts.values,
        names=counts.index,
        title=f'Pie Chart for {selected_column}',
        template='plotly_dark',
        color=counts.index,  # Set color based on categories
        color_discrete_sequence=px.colors.sequential.Aggrnyl_r  # Use a qualitative color sequence
    )
    hist_fig = px.histogram(
        filtered_df, x=hist_column,
        title=f'Histogram of {hist_column}',
        template='plotly_dark',
        #color=counts.index,
        color_discrete_sequence=px.colors.sequential.Aggrnyl_r
    )

    # Box plot for the selected histogram column
    box_fig = px.box(
        filtered_df, x=hist_column,
        title=f'Box Plot of {hist_column}',
        template='plotly_dark',
        #color=counts.index,
        color_discrete_sequence=px.colors.sequential.Aggrnyl_r
    )


    return scatter_fig, bar_fig, pie_fig, hist_fig, box_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>